In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

import resnet_v2
import tensorflow as tf
from training import get_labels_from_annotation_batch
slim = tf.contrib.slim
from scipy.misc import imread
import numpy as np
from tensorflow.python.ops import control_flow_ops
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
x = tf.placeholder(tf.float32, shape=[None, 784])
batch_images = tf.reshape(x, [-1, 28,28,1])

y_ = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32, name="dropout_keep_probability")
is_training = tf.placeholder(tf.bool, name="batch_norm_switch")

In [3]:
k=12
theta=1.0 # compression_factor
number_of_classes = 10
batch_norm_decay = 0.997
batch_norm_epsilon = 1e-5,
batch_norm_scale = True

In [4]:
@slim.add_arg_scope
def block_unit(net, k, id, keep_prob=1.0):
    # block_unit implements a bottleneck layer 
    with tf.variable_scope("block_unit_" + str(id)):
        
        # each 1×1 convolution produce 4k feature-maps
        net = slim.conv2d(net, 4*k, [1,1], scope="conv1", activation_fn=None, normalizer_fn=None)
        net = slim.dropout(net, keep_prob, scope='dropout1')
        net = slim.batch_norm(net, activation_fn=tf.nn.relu)

        net = slim.conv2d(net, k, [3,3], scope="conv3x3", activation_fn=None, normalizer_fn=None)
        net = slim.dropout(net, keep_prob, scope='dropout2')
        net = slim.batch_norm(net, activation_fn=tf.nn.relu)
        return net

In [5]:
@slim.add_arg_scope
def add_transition_layer(net, id):
    # We use 1×1 convolution followed by 2×2 average pooling as transition layers between two contiguous dense blocks
    with tf.variable_scope("transition_layer_" + str(id)):  
        current_depth = slim.utils.last_dimension(net.get_shape(), min_rank=4)

        net = slim.batch_norm(net, activation_fn=tf.nn.relu)
        net = slim.conv2d(net, theta*current_depth, [1,1], scope='conv1x1', 
                          activation_fn=None, normalizer_fn=None)
        net = slim.dropout(net, keep_prob, scope='dropout')
        net = slim.avg_pool2d(net, [2,2], scope='avg_pool', stride=2)
        return net

In [6]:
batch_norm_params = {
  'decay': batch_norm_decay,
  'epsilon': batch_norm_epsilon,
  'scale': batch_norm_scale,
  'is_training': is_training,
  'updates_collections': tf.GraphKeys.UPDATE_OPS,
}

# batch_images shape (?, 224,224,3)
with slim.arg_scope([slim.conv2d], padding='SAME',
                    weights_initializer=slim.variance_scaling_initializer(),
                    activation_fn=tf.nn.relu,
                    stride=1,
                    normalizer_fn=slim.batch_norm,
                    normalizer_params=batch_norm_params,
                    weights_regularizer=slim.l2_regularizer(0.0001)):

    with slim.arg_scope([slim.batch_norm], **batch_norm_params):

        # We refer to layers between blocks as transition layers, which do convolution and pooling
        # net = slim.conv2d(batch_images, 2*k, [7,7], scope='conv1')
        # net = slim.max_pool2d(net, [3,3], scope='pool1') # (?,56,56,2*k)

        net = slim.conv2d(batch_images, 2*k, [7,7], scope='conv1')

        with tf.variable_scope("dense_block_1"):
            unit1 = block_unit(net, k, 1)

            unit2 = block_unit(unit1, k, 2)
            unit1_2 = tf.concat((unit1, unit2), axis=3, name="unit1_2")

            unit3 = block_unit(unit1_2, k, 3)
            unit1_2_3 = tf.concat((unit1_2, unit3), axis=3, name="unit1_2_3")

            unit4 = block_unit(unit1_2_3, k, 4)
            block_1 = tf.concat((unit1_2_3, unit4), axis=3, name="unit1_2_3_4")
        
        net = add_transition_layer(block_1, 1)
        
        with tf.variable_scope("dense_block_2"):
            unit1 = block_unit(net, k, 1)

            unit2 = block_unit(unit1, k, 2)
            unit1_2 = tf.concat((unit1, unit2), axis=3, name="unit1_2")

            unit3 = block_unit(unit1_2, k, 3)
            unit1_2_3 = tf.concat((unit1_2, unit3), axis=3, name="unit1_2_3")

            unit4 = block_unit(unit1_2_3, k, 4)
            block_2 = tf.concat((unit1_2_3, unit4), axis=3, name="unit1_2_3_4")
        
        net = add_transition_layer(block_2, 2)
        
        with tf.variable_scope("dense_block_3"):
            unit1 = block_unit(net, k, 1)

            unit2 = block_unit(unit1, k, 2)
            unit1_2 = tf.concat((unit1, unit2), axis=3, name="unit1_2")

            unit3 = block_unit(unit1_2, k, 3)
            unit1_2_3 = tf.concat((unit1_2, unit3), axis=3, name="unit1_2_3")

            unit4 = block_unit(unit1_2_3, k, 4)
            block_3 = tf.concat((unit1_2_3, unit4), axis=3, name="unit1_2_3_4")
        
        print(block_3)
        net = slim.batch_norm(block_3, activation_fn=tf.nn.relu)
        
        # Flatten the feature map by global average pooling
        global_avg_pooling = tf.reduce_mean(net, (1, 2))
        logits = slim.fully_connected(global_avg_pooling, number_of_classes, scope='fully_connected', 
                                      activation_fn=None, normalizer_fn=None)

Tensor("dense_block_1/unit1_2_3_4:0", shape=(?, 28, 28, 48), dtype=float32)
Tensor("transition_layer_1/avg_pool/AvgPool:0", shape=(?, 14, 14, 48), dtype=float32)
Tensor("dense_block_2/unit1_2_3_4:0", shape=(?, 14, 14, 48), dtype=float32)
Tensor("transition_layer_2/avg_pool/AvgPool:0", shape=(?, 7, 7, 48), dtype=float32)


In [7]:
batch_size = 64
epochs = 40
initial_lr = 0.1
second_lr = initial_lr/10
third_lr = second_lr/10

In [8]:
global_step = tf.Variable(0, trainable=False)
boundaries = [1280, 1920]
values = [initial_lr, second_lr, third_lr]
learning_rate = tf.train.piecewise_constant(global_step, boundaries, values)

In [9]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=logits))

optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
train_step = slim.learning.create_train_op(cross_entropy, optimizer, global_step=global_step)

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
if update_ops:
    updates = tf.group(*update_ops)
    cross_entropy = control_flow_ops.with_dependencies([updates], cross_entropy)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(2000):
        batch = mnist.train.next_batch(batch_size)

        _, loss = sess.run([train_step, cross_entropy], feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0, is_training: True})

        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={
              x: batch[0], y_: batch[1], keep_prob: 1.0, is_training: False})

            print("Step:", i, "Loss:", loss, "Training accuracy:", train_accuracy )

    print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0,  is_training: False}))

Step: 0 Loss: 2.337 Training accuracy: 0.0625
Step: 100 Loss: 0.138563 Training accuracy: 0.078125
Step: 200 Loss: 0.171861 Training accuracy: 0.078125
Step: 300 Loss: 0.0674317 Training accuracy: 0.109375
Step: 400 Loss: 0.0968171 Training accuracy: 0.78125
Step: 500 Loss: 0.117412 Training accuracy: 0.671875
Step: 600 Loss: 0.276976 Training accuracy: 0.515625
Step: 700 Loss: 0.232482 Training accuracy: 0.859375
Step: 800 Loss: 0.103286 Training accuracy: 0.765625
Step: 900 Loss: 0.159462 Training accuracy: 0.796875
Step: 1000 Loss: 0.0170348 Training accuracy: 0.953125
Step: 1100 Loss: 0.0238303 Training accuracy: 0.921875
Step: 1200 Loss: 0.083677 Training accuracy: 0.875
Step: 1300 Loss: 0.0357825 Training accuracy: 0.9375
Step: 1400 Loss: 0.0044147 Training accuracy: 1.0
Step: 1500 Loss: 0.00961189 Training accuracy: 0.984375
Step: 1600 Loss: 0.00329354 Training accuracy: 1.0
Step: 1700 Loss: 0.00616657 Training accuracy: 1.0
Step: 1800 Loss: 0.0013633 Training accuracy: 1.0
Step